# Django RESTful API with a legacy database

This document explains how to develop a RESTful API on a legacy database with Django.
For the requirements and setup of this workshop, read the README.md document in the repository.

## Project initialization

Let's create the project and application with Django tooling:

    django-admin.py startproject url_shortener .
    python manage.py statapp api

## Setup application

Modify `url_shortener/settings.py`, so it looks like this:

    INSTALLED_APPS = [
        'django.contrib.admin',
        'django.contrib.auth',
        'django.contrib.contenttypes',
        'django.contrib.sessions',
        'django.contrib.messages',
        'django.contrib.staticfiles',
        'api',
    ]

## Generate models from legacy database

Let's create the models inferring these from current database:

    python manage.py inspectdb > api/models.py

## Prepare API application to use legacy models

Remove all `managed = False` from `api/models.py`.

Create admin features in `api/admin.py`:

    from api.models import Urls, Hits
    from django.contrib import admin


    admin.site.register(Urls)
    admin.site.register(Hits)

## Initialize application

Let's initialize the Django database objects (admin, etc.):

    python manage.py migrate
    
Now, fake the initialization of the derived models for api:

    python manage.py makemigrations api
    python manage.py migrate --fake-initial api
    
Let's create a superuser account:

    python manage.py createsuperuser

## Preliminar test

    python manage.py runserver 8080

Go to http://127.0.0.1:8080/admin, login as admin, and create a URL and a Hit.
This should work more or less fine, ...


but there are some issues:

* The models are pluralized in a bad way.

* The references to their objects, are names like `Urlss object`, not giving a clue of what's the object referencing to.

* The fields are not automatically incremented.

## Fixing small issues here

### Pluralized model names

Add `verbose_name = '...'` to each models' `Meta` subclass.

### Object names in references

Add `__str__` method with some code similar to this:

    def __str__(self):
        return 'Url for {}'.format(self.url)

### Autoincrement fields not being used appropriately

Change the `id` fields to be `AutoField`s.

## Create the API

Create `api/serializers.py` :

    from api.models import Urls, Hits
    from rest_framework import serializers


    class UrlSerializer(serializers.HyperlinkedModelSerializer):
        class Meta:
            model = Urls
            fields = ('url', 'short_code', 'ip', 'time')


    class HitSerializer(serializers.HyperlinkedModelSerializer):
        class Meta:
            model = Hits
            fields = ('url', 'ip', 'time')

Add the following code to `api/views.py`:

    from api.models import Urls, Hits
    from rest_framework import viewsets
    from api.serializers import UrlSerializer, HitSerializer


    class UrlViewSet(viewsets.ModelViewSet):
        """
        API endpoint that allows urls to be viewed or edited.
        """
        queryset = Urls.objects.all().order_by('-time')
        serializer_class = UrlSerializer


    class HitViewSet(viewsets.ModelViewSet):
        """
        API endpoint that allows hits to be viewed or edited.
        """
        queryset = Hits.objects.all()
        serializer_class = HitSerializer

Modify `url_shortener/urls.py`, to get this code:

    from django.conf.urls import url, include
    from rest_framework import routers
    from django.contrib import admin
    from api import views


    router = routers.DefaultRouter()
    router.register(r'urls', views.UrlViewSet)
    router.register(r'hits', views.HitViewSet)

    urlpatterns = [
        url(r'^admin/', admin.site.urls),
        url(r'^', include(router.urls)),
        url(r'^api-auth/', include('rest_framework.urls', namespace='rest_framework')),
    ]

Modify `url_shortener/settings.py`:

    INSTALLED_APPS = [
        'django.contrib.admin',
        'django.contrib.auth',
        'django.contrib.contenttypes',
        'django.contrib.sessions',
        'django.contrib.messages',
        'django.contrib.staticfiles',
        'api',
        'rest_framework',
    ]

    REST_FRAMEWORK = {
        'DEFAULT_PERMISSION_CLASSES': ('rest_framework.permissions.IsAdminUser',),
        'PAGE_SIZE': 10
    }


## Test it

    python manage.py runserver 8080
    http -a admin:1234admin5678 http://127.0.0.1:8080/urls/
    http --form -a admin:1234admin5678 POST http://127.0.0.1:8080/urls/ \
         url=https://devex.com \
         ip=127.0.0.1 \
         short_code=devex \
         time=2016-05-14

## Make it public read-only

Change `url_shortener/settings.py`:

    REST_FRAMEWORK = {
        'DEFAULT_PERMISSION_CLASSES': ('rest_framework.permissions.IsAuthenticatedOrReadOnly',),
        'PAGE_SIZE': 10
    }


## Test it again

    python manage.py runserver 8080
    http http://127.0.0.1:8080/urls/
    http --form -a admin:1234admin5678 POST http://127.0.0.1:8080/urls/ \
         url=https://devex.com \
         ip=127.0.0.1 \
         short_code=devex \
         time=2016-05-14

## Want more? Homework?

Modify implementation on Hit model, so time is not asked, and it gets the time of the request on model save.

Let's implement the logic for adding URLs, with optional `short_code`, so if it is not present